In [1]:
import pandas as pd
import numpy as np
import re
import pymorphy2
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

### Создать DF с вакансиями

In [2]:
vacancy_df = pd.read_csv('data_0402.csv')

In [3]:
vacancy_df.department.value_counts(normalize=True)*100

Сбер для экспертов                 49.885220
Сбер. IT                           25.392901
Сбер. Начало карьеры               21.561010
Сбер. Работа с клиентами            1.289069
Сбер. Экспертам и руководителям     1.218435
Сбер. Data Science                  0.565072
Сбер. Кибербезопасность             0.088292
Name: department, dtype: float64

# Предобработка данных

In [3]:
# удалить  тэги из описаний
pattern = r'</*[a-z]+>'
vacancy_df.description = vacancy_df.description.apply(lambda x: re.sub(pattern, ' ', x))

# удалить лишние пробелы
pattern = r'\s{1}\s+'
vacancy_df.description = vacancy_df.description.apply(lambda x: re.sub(pattern, ' ', x))

# # заменить слэш на пробелы (в перечислении вариантов, например: SSL/TLS/mTLS, MySQL/Postgres)
# лишнее, т.к. отрабатывается в функции ниже
# pattern = r'(\S)(/)+(\S)'
# vacancy_df.description = vacancy_df.description.apply(lambda x: re.sub(pattern, r'\1 \3', x))

# заменить "ё" на "е"
pattern = r'<[Ёё]+>'
vacancy_df.description = vacancy_df.description.apply(lambda x: re.sub(pattern, 'е', x))

# заменить ошибочную английскую 'c' в начале слов на русскую 'с'
pattern = r'(c)([а-я]{2,})'
vacancy_df.description = vacancy_df.description.apply(lambda x: re.sub(pattern, r'с\2', x))
# заменить ошибочную русскую 'с' на английскую 'c' в начале слов 
pattern = r'(с)([a-z]{2,})'
vacancy_df.description = vacancy_df.description.apply(lambda x: re.sub(pattern, r'c\2', x))

In [4]:
# перевести даты создания и публикации в datetime
vacancy_df['created_at'] = pd.to_datetime(vacancy_df['created_at'])
vacancy_df['published_at'] = pd.to_datetime(vacancy_df['published_at'])

# ЧЕРНОВИК

In [6]:
df = vacancy_df.head(1000).copy()

## 1. ПРЕДОБРАБОТКА ОПИСАНИЙ

Очистка от знаков препинания

In [7]:
# удалить знаки пунктуации
def remove_trash(string): 
    pattern = r'[^А-Яа-яA-Za-z0-9+]+'
    try:
      string = re.sub(pattern, ' ', string)
    except Exception as e:
      print(e)
    return string

In [8]:
df['desc_clean'] = df.description.apply(remove_trash)

Создание списка стоп-слов

In [9]:
# стоп слова
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Artur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
stop_words_ru = stopwords.words('russian')

In [10]:
stop_words_ru.extend(['вакансия', 'каждый', 'который', 'команда', 'любой', 'наш', 'самый', 'сбер', 'сбербанк', 'сберпрайм', 'сберпрайм+', 'сбербизнес', 'свой', 'экосистема', 'подразделение', 'интересный', 'работодатель', 'хороший', 'откликнуться', 'детский', 'родитель'])

In [11]:
# часто вречающийся общий текст в описании (разбить на слова и добавить в stop_words)
string = 'подготовить ты ипотека выгодный каждый сотрудник льготный условие первый льготный страхование близкий бесплатный подписка сберпрайм+ скидка продукт компания партнёр okko сбер маркет delivery club самокат ситимобила сбер еаптека другой обучение счёт компания онлайн курс виртуальный школа возможность получить новый квалификация курс будущий родитель материальный поддержка молодой детский отдых подарок счёт компания реферальный программа сотрудник можно пригласить команда знакомый профессионал получить вознаграждение рубль корпоративный пенсионный программа интересный вакансия откликнуться записать видеоинтервью ссылка письмо любой удобный устройство пройти собеседование'
string = string.split()
stop_words_ru.extend(string)
stop_words_ru = list(set(stop_words_ru))

нормализация описания (избавление от различных форм слов)

In [12]:
# лемматизация
morph = pymorphy2.MorphAnalyzer()

In [15]:
# функция лемматизации
def lemmatize(data, col=None):
    t = []
    if type(data) == pd.core.series.Series:
        text = data[col]
    else:
        text = data
    for word in text.split():
        if len(word)<=3:
            continue
        p = morph.parse(word)[0]
        t.append(p.normal_form)
    return " ".join(t)

In [16]:
df['desc_normal'] = df.apply(lemmatize, axis=1, col='desc_clean')

In [ ]:
# исключить вакансии на английском
# примитивный вариант поиска вакансий на английском

# pattern = r'[^А-Яа-я]+\s+[^А-Яа-я]+\s+[^А-Яа-я]+\s+[^А-Яа-я]+\s+[^А-Яа-я]+\s+[^А-Яа-я]+[^А-Яа-я]+\s+[^А-Яа-я]'
# arr = []
# for i in df['desc_normal']:
#     test = re.match(pattern, i)
#     if test != None:
#         arr.append(i)
# arr

## 2. ТОКЕНИЗАЦИЯ ОПИСАНИЙ

### МЕТОД 1. Граф векторизатор

In [19]:
vec_CV = CountVectorizer(stop_words=stop_words_ru, min_df=3)

In [20]:
desc_vectors_CV = vec_CV.fit_transform(df['desc_normal'])

In [21]:
print('Vocabulary length:', len(vec_CV.get_feature_names()))

Vocabulary length: 3143


Добавление в DF векторных представлений описаний

In [22]:
df['CV_vec'] = list(desc_vectors_CV.toarray())

Предобработка опыта

In [23]:
expirience = '''Консультирование и продажа банковских продуктов: открытие вкладов, кредитов, кредитных и дебетовых карт, \
подключение автоплатежей, продажа инвестиционных продуктов, переводы и т.д.'''

In [24]:
# удаление знаков препинания
expirience = remove_trash(expirience)
expirience

'Консультирование и продажа банковских продуктов открытие вкладов кредитов кредитных и дебетовых карт подключение автоплатежей продажа инвестиционных продуктов переводы и т д '

In [25]:
# нормализация
expirience = lemmatize(expirience)
expirience

'консультирование продажа банковский продукт открытие вклад кредит кредитный дебетовый карта подключение автоплатёж продажа инвестиционный продукт перевод'

Токенизация опыта из резюме

In [27]:
vec_exp_CV = CountVectorizer(stop_words=stop_words_ru, vocabulary=vec_CV.get_feature_names())

In [28]:
exp_vector_CV = vec_exp_CV.fit_transform([expirience])

Определение косинусного сходства опыта с описаниями вакансий

In [29]:
# функция для определения косинусного ходства двух векторов
def cosine_sim_vectors(vec1, vec2):    
    return cosine_similarity([vec1, vec2])[1][0]

In [30]:
df['CV_cosine'] = df['CV_vec'].apply(cosine_sim_vectors, vec2=exp_vector_CV.toarray()[0])

In [32]:
top_10_CV = df.sort_values(by='CV_cosine', ascending=False).head(10)

In [34]:
top_10_CV[["name", "description", "specializations"]]

,name,description,specializations
866,Старший клиентский менеджер отдела по работе с...,Обязанности: формирование потребности и разви...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
757,Руководитель Премьер офиса,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
986,Руководитель Премьер офиса,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
694,Финансовый советник,Обязанности: вести инвестиционный портфель кл...,"['Инвестиционная компания', 'Прямые инвестиции..."
777,Консультант по работе с Пенсионным Фондом России,Обязанности: - организация качественного пров...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
39,Территориальный менеджер по продуктам глобальн...,Обязанности: поиск и отбор клиентов для прода...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
591,Финансовый советник,Сбербанк Первый – это офис премиального обслу...,"['Методология, Банковские технологии', 'Разраб..."
192,Клиентский менеджер Сбербанк Первый,Сбербанк Первый – это офис премиального обслу...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
653,Клиентский менеджер (Премьер),Если ты: имеешь опыт успешных продаж банковск...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
817,Руководитель ВИП ВСП,Сбербанк Первый – это офис премиального обслу...,"['Розничная торговля', 'Прямые продажи', 'Мене..."


### МЕТОД 2. TfidfVectorizer

In [35]:
vec_TV = TfidfVectorizer(stop_words=stop_words_ru, min_df=3)

In [36]:
desc_vectors_Tfid = vec_TV.fit_transform(df['desc_normal'])

In [37]:
df['TV_vec'] = list(desc_vectors_Tfid.toarray())

In [39]:
# токенизация описания опыта
vec_exp_TV = TfidfVectorizer(stop_words=stop_words_ru, vocabulary=vec_TV.get_feature_names())

In [40]:
exp_vector_TV = vec_exp_TV.fit_transform([expirience])

In [41]:
sum(exp_vector_TV.toarray()[0])

3.2071349029490923

In [42]:
sum(exp_vector_CV.toarray()[0])

12

In [43]:
df['TV_cosine'] = df['TV_vec'].apply(cosine_sim_vectors, vec2=exp_vector_TV.toarray()[0])

In [44]:
top_10_TV = df.sort_values(by='TV_cosine', ascending=False).head(10)

In [45]:
top_10_TV[["name", "area", "salary", "description", "specializations"]]

,name,area,salary,description,specializations
757,Руководитель Премьер офиса,Уфа,NaN,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
986,Руководитель Премьер офиса,Урай,NaN,Обязанности: организация работы ВИП офиса в с...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
288,Клиентский менеджер Сбер Премьер в МИДДЛ-офис,Самара,60100.0,Обязанности: - обслуживание премиальных клиен...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
866,Старший клиентский менеджер отдела по работе с...,Волгоград,65000.0,Обязанности: формирование потребности и разви...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
770,Старший клиентский менеджер,Шадринск,36600.0,Если ты хочешь: построить карьеру в банковско...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
270,Старший клиентский менеджер,Сургут,93500.0,Если ты хочешь: построить карьеру в банковско...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
856,Старший клиентский менеджер,Когалым,89600.0,Если ты хочешь: построить карьеру в банковско...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
817,Руководитель ВИП ВСП,Ханты-Мансийск,178000.0,Сбербанк Первый – это офис премиального обслу...,"['Розничная торговля', 'Прямые продажи', 'Мене..."
777,Консультант по работе с Пенсионным Фондом России,Красноярск,38500.0,Обязанности: - организация качественного пров...,"['Прямые продажи', 'Оптовая торговля', 'Менедж..."
694,Финансовый советник,Южно-Сахалинск,NaN,Обязанности: вести инвестиционный портфель кл...,"['Инвестиционная компания', 'Прямые инвестиции..."
